In [29]:
import warnings
from pandas.errors import SettingWithCopyWarning  # <- import the class

warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=SettingWithCopyWarning)
warnings.simplefilter('ignore', category=UserWarning)
import warnings
warnings.filterwarnings('ignore')


In [19]:
import numpy as np

In [1]:
import pandas as pd

# Load the cleaned dataset
trips = pd.read_parquet("final_cleaned_trips_20250826.parquet")

In [3]:
trips1=trips.copy()

In [5]:
features = [
    'Trip Seconds',
    'Trip Miles',
    'Pickup Community Area',
    'pct_low_income',
    'pct_black',
    'pct_hispanic',
    'pct_nonwhite',
    'start_hour',
    'day_of_week',
    'Additional Charges'
]
target = 'Fare'  


In [9]:
print(trips1['Shared Trip Authorized'].value_counts(dropna=False))


Shared Trip Authorized
False    12836253
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split

# Select features and target
X = trips1[features]
y = trips1[target]

# Drop rows with missing values in X
X = X.dropna()
y = y.loc[X.index] 

# One-hot encode 'day_of_week'
X_encoded = pd.get_dummies(X, columns=['day_of_week'], drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)



### Step1 Building models:Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R² Score:", r2_score(y_test, y_pred))


RMSE: 6.425435295682857
R² Score: 0.696597551464203


### HistGradientBoostingRegressor

In [21]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 1) Downcast to float32 to halve memory
X_train32 = X_train.astype(np.float32)
X_test32  = X_test.astype(np.float32)


hgb = HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.1,
    max_depth=8,            # control complexity
    max_leaf_nodes=31,      # tree size
    max_bins=255,           # histogram bins
    l2_regularization=1.0,  # stability
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)

hgb.fit(X_train32, y_train)
pred_hgb = hgb.predict(X_test32)

rmse_hgb = mean_squared_error(y_test, pred_hgb, squared=False)
r2_hgb = r2_score(y_test, pred_hgb)
print(f"HistGB RMSE: {rmse_hgb:.4f} | R²: {r2_hgb:.4f}")


HistGB RMSE: 5.8041 | R²: 0.7524


In [23]:
from sklearn.metrics import r2_score, root_mean_squared_error

def eval_regression(y_true, y_pred, name="model"):
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} → RMSE: {rmse:.4f} | R²: {r2:.4f}")
    return rmse, r2


In [25]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score, root_mean_squared_error
import numpy as np

def eval_reg(y_true, y_pred):
    return root_mean_squared_error(y_true, y_pred), r2_score(y_true, y_pred)

candidates = [
    dict(max_depth=6,  max_leaf_nodes=31,  learning_rate=0.07, min_samples_leaf=20, l2_regularization=1.0),
    dict(max_depth=8,  max_leaf_nodes=63,  learning_rate=0.05, min_samples_leaf=50, l2_regularization=1.0),
    dict(max_depth=8,  max_leaf_nodes=31,  learning_rate=0.05, min_samples_leaf=20, l2_regularization=0.5),
    dict(max_depth=10, max_leaf_nodes=63,  learning_rate=0.05, min_samples_leaf=100, l2_regularization=2.0),
    dict(max_depth=6,  max_leaf_nodes=127, learning_rate=0.03, min_samples_leaf=50, l2_regularization=1.0),
]

best = None
for i, params in enumerate(candidates, 1):
    hgb = HistGradientBoostingRegressor(
        loss="squared_error",
        early_stopping=True, validation_fraction=0.1,
        max_bins=255, max_iter=500, random_state=42,
        **params
    )
    hgb.fit(X_train32, y_train)
    pred = hgb.predict(X_test32)
    rmse, r2 = eval_reg(y_test, pred)
    print(f"[{i}] {params} → RMSE {rmse:.4f} | R² {r2:.4f}")
    if (best is None) or (rmse < best["rmse"]):
        best = dict(model=hgb, params=params, rmse=rmse, r2=r2)

print("\nBest:", best["params"], "→ RMSE", round(best["rmse"],4), "| R²", round(best["r2"],4))
best_hgb = best["model"]


[1] {'max_depth': 6, 'max_leaf_nodes': 31, 'learning_rate': 0.07, 'min_samples_leaf': 20, 'l2_regularization': 1.0} → RMSE 5.6971 | R² 0.7615
[2] {'max_depth': 8, 'max_leaf_nodes': 63, 'learning_rate': 0.05, 'min_samples_leaf': 50, 'l2_regularization': 1.0} → RMSE 5.6509 | R² 0.7653
[3] {'max_depth': 8, 'max_leaf_nodes': 31, 'learning_rate': 0.05, 'min_samples_leaf': 20, 'l2_regularization': 0.5} → RMSE 5.7079 | R² 0.7606
[4] {'max_depth': 10, 'max_leaf_nodes': 63, 'learning_rate': 0.05, 'min_samples_leaf': 100, 'l2_regularization': 2.0} → RMSE 5.6326 | R² 0.7668
[5] {'max_depth': 6, 'max_leaf_nodes': 127, 'learning_rate': 0.03, 'min_samples_leaf': 50, 'l2_regularization': 1.0} → RMSE 5.7417 | R² 0.7577

Best: {'max_depth': 10, 'max_leaf_nodes': 63, 'learning_rate': 0.05, 'min_samples_leaf': 100, 'l2_regularization': 2.0} → RMSE 5.6326 | R² 0.7668


Fast, model‑agnostic interpretability

These help you explain drivers of fares before we go into fairness slicing.

##### Permutation importance

In [27]:
import numpy as np
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, mean_squared_error

# 1) Use EXACT columns & order the model was trained on
try:
    fit_cols = list(best_hgb.feature_names_in_)   # works if you fit on a DataFrame
except AttributeError:
    fit_cols = list(X_test32.columns)            # fallback

# 2) Sample rows for speed (adjust N if still slow)
N = min(2000, len(X_test32))
X_sub = X_test32.loc[:, fit_cols].sample(n=N, random_state=42)
y_sub = y_test.loc[X_sub.index]

# 3) Fast permutation importance on ALL features (same schema as fit)
rmse_scorer = make_scorer(mean_squared_error, squared=False, greater_is_better=False)

r = permutation_importance(
    best_hgb,
    X_sub,                  
    y_sub,
    n_repeats=2,           
    random_state=42,
    n_jobs=1,
    scoring=rmse_scorer,
    max_samples=1000,       
)

pi = pd.Series(r.importances_mean, index=fit_cols).sort_values(ascending=False)

features_of_interest = [
    "Trip Miles", "Trip Seconds", "Shared Trip Authorized", "Additional Charges",
    "pct_low_income", "pct_black", "pct_hispanic", "pct_nonwhite",
    "dropoff_pct_low_income", "dropoff_pct_black",
    "dropoff_pct_hispanic", "dropoff_pct_nonwhite",
]
to_show = [f for f in features_of_interest if f in pi.index]

print("\n=== Top 15 overall (by RMSE drop) ===")
print(pi.head(15).round(6))

print("\n=== Selected features (by RMSE drop) ===")
print(pi.loc[to_show].round(6).sort_values(ascending=False))



=== Top 15 overall (by RMSE drop) ===
Trip Miles               5.338393
Trip Seconds             1.820264
Additional Charges       1.151791
start_hour               0.938428
pct_nonwhite             0.299119
Pickup Community Area    0.290390
day_of_week_Monday       0.289240
pct_black                0.268964
pct_low_income           0.229094
day_of_week_Sunday       0.220911
pct_hispanic             0.182222
day_of_week_Saturday     0.182140
day_of_week_Tuesday      0.181924
day_of_week_Thursday     0.166831
day_of_week_Wednesday    0.153948
dtype: float64

=== Selected features (by RMSE drop) ===
Trip Miles            5.338393
Trip Seconds          1.820264
Additional Charges    1.151791
pct_nonwhite          0.299119
pct_black             0.268964
pct_low_income        0.229094
pct_hispanic          0.182222
dtype: float64


#### Step 2: Fairness evaluation
Bias_mean(y_true − y_pred): positive means underprediction; negative means overprediction. Persistent signed bias for certain groups is a red flag.

In [31]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# Make a working frame
df_eval = pd.DataFrame({
    "y_true": y_test,
    "y_hat_hgb": pred_hgb,  # from the tuned model above (or use your earlier pred_hgb)
}, index=X_test32.index)

# Attach the grouping attributes from X (or original df) using the test indices
cols_for_groups = ["pct_low_income", "pct_black", "pct_hispanic", "pct_nonwhite"]
G = trips1.loc[df_eval.index, cols_for_groups].copy()

# Define helpers
def bucketize(series, quantiles=(0.25, 0.5, 0.75)):
    qs = series.quantile(list(quantiles))
    return pd.cut(series, bins=[-np.inf] + qs.tolist() + [np.inf],
                  labels=[f"Q{i}" for i in range(1, len(qs)+2)])

def group_error_table(y_true, y_pred, group_series, group_name):
    tmp = pd.DataFrame({"y_true": y_true, "y_pred": y_pred, group_name: group_series})
    agg = tmp.groupby(group_name).apply(
        lambda d: pd.Series({
            "n": len(d),
            "MAE": mean_absolute_error(d["y_true"], d["y_pred"]),
            "RMSE": root_mean_squared_error(d["y_true"], d["y_pred"]),
            "Bias_mean(y_true - y_pred)": (d["y_true"] - d["y_pred"]).mean()
        })
    )
    # Simple disparity metrics
    agg["RMSE_gap_vs_Q1"] = agg["RMSE"] - agg["RMSE"].iloc[0]  # assuming Q1 exists
    return agg.round(4)

results = {}
for attr in ["pct_low_income", "pct_black"]:
    bucket = bucketize(G[attr])
    results[attr] = group_error_table(df_eval["y_true"], df_eval["y_hat_hgb"], bucket, f"{attr}_bucket")

# Show
for k, v in results.items():
    print(f"\n=== Group error by {k} (quantile buckets) ===")
    display(v)



=== Group error by pct_low_income (quantile buckets) ===


,n,MAE,RMSE,Bias_mean(y_true - y_pred),RMSE_gap_vs_Q1
pct_low_income_bucket,,,,,
Q1,978594.0,3.7985,5.9664,0.0099,0.0000
Q2,370945.0,3.3630,5.6588,-0.0148,-0.3076
Q3,588116.0,3.8893,6.9456,-0.0414,0.9792
Q4,628493.0,2.5204,4.2520,0.0258,-1.7143



=== Group error by pct_black (quantile buckets) ===


,n,MAE,RMSE,Bias_mean(y_true - y_pred),RMSE_gap_vs_Q1
pct_black_bucket,,,,,
Q1,643566.0,3.2157,5.3768,-0.0291,0.0000
Q2,733666.0,4.1226,6.8992,0.0403,1.5224
Q3,595795.0,3.6206,5.9349,0.0009,0.5581
Q4,593121.0,2.6720,4.4858,-0.0258,-0.8910


### Evaluate fairness across groups 👩🏾‍🤝‍👩🏼.
We’ll compare Linear Regression (baseline) vs. your tuned HistGB model and check whether prediction errors differ across income and race quartiles.

#### Step 1: Predictions from both models

In [33]:
from sklearn.linear_model import LinearRegression

# Linear Regression baseline
linreg = LinearRegression()
linreg.fit(X_train, y_train)
pred_lr = linreg.predict(X_test)

# HistGB tuned model (already trained)
pred_hgb = best_hgb.predict(X_test32)


#### Step 2: Fairness evaluation function

This computes MAE, RMSE, and bias for each quartile of a grouping variable:

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

def bucketize(series, quantiles=(0.25, 0.5, 0.75)):
    qs = series.quantile(list(quantiles))
    return pd.cut(series, bins=[-np.inf] + qs.tolist() + [np.inf],
                  labels=[f"Q{i}" for i in range(1, len(qs)+2)])

def group_error_table(y_true, y_pred, group_series, group_name):
    tmp = pd.DataFrame({"y_true": y_true, "y_pred": y_pred, group_name: group_series})
    agg = tmp.groupby(group_name).apply(
        lambda d: pd.Series({
            "n": len(d),
            "MAE": mean_absolute_error(d["y_true"], d["y_pred"]),
            "RMSE": root_mean_squared_error(d["y_true"], d["y_pred"]),
            "Bias (y_true - y_pred)": (d["y_true"] - d["y_pred"]).mean()
        })
    )
    agg["RMSE_gap_vs_Q1"] = agg["RMSE"] - agg["RMSE"].iloc[0]  # disparity vs lowest group
    return agg.round(4)


#### Step 3: Run fairness evaluation for both models

In [37]:
results = {}

for model_name, preds in [("Linear Regression", pred_lr), ("HistGB", pred_hgb)]:
    for attr in ["pct_low_income", "pct_black"]:
        bucket = bucketize(trips1.loc[y_test.index, attr])
        table = group_error_table(y_test, preds, bucket, f"{attr}_bucket")
        results[(model_name, attr)] = table

# Show results
for (model, attr), table in results.items():
    print(f"\n=== {model}: Group error by {attr} (quartiles) ===")
    display(table)



=== Linear Regression: Group error by pct_low_income (quartiles) ===


,n,MAE,RMSE,Bias (y_true - y_pred),RMSE_gap_vs_Q1
pct_low_income_bucket,,,,,
Q1,978594.0,4.2618,6.6022,0.1051,0.0000
Q2,370945.0,3.8686,6.2411,-0.0914,-0.3611
Q3,588116.0,4.4296,7.4494,0.0565,0.8471
Q4,628493.0,3.4364,5.0778,-0.1556,-1.5245



=== Linear Regression: Group error by pct_black (quartiles) ===


,n,MAE,RMSE,Bias (y_true - y_pred),RMSE_gap_vs_Q1
pct_black_bucket,,,,,
Q1,643566.0,3.9491,6.1380,-0.4688,0.0000
Q2,733666.0,4.4396,7.2925,0.4693,1.1546
Q3,595795.0,4.1645,6.6520,0.1389,0.5140
Q4,593121.0,3.5249,5.2455,-0.2039,-0.8925



=== HistGB: Group error by pct_low_income (quartiles) ===


,n,MAE,RMSE,Bias (y_true - y_pred),RMSE_gap_vs_Q1
pct_low_income_bucket,,,,,
Q1,978594.0,3.6650,5.7950,0.0044,0.0000
Q2,370945.0,3.2614,5.5157,-0.0093,-0.2793
Q3,588116.0,3.7494,6.7316,-0.0143,0.9366
Q4,628493.0,2.4428,4.1101,0.0050,-1.6849



=== HistGB: Group error by pct_black (quartiles) ===


,n,MAE,RMSE,Bias (y_true - y_pred),RMSE_gap_vs_Q1
pct_black_bucket,,,,,
Q1,643566.0,3.0929,5.2069,-0.0000,0.0000
Q2,733666.0,3.9985,6.7152,-0.0006,1.5082
Q3,595795.0,3.4968,5.7482,-0.0065,0.5412
Q4,593121.0,2.5783,4.3450,-0.0002,-0.8620


In [39]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor

# -------- 1) Prep in place: dtypes + derived columns
df = trips1 

# Ensure datetimes + time features
if not np.issubdtype(df["Trip Start Timestamp"].dtype, np.datetime64):
    df["Trip Start Timestamp"] = pd.to_datetime(df["Trip Start Timestamp"], errors="coerce")
if "start_hour" not in df.columns:
    df["start_hour"] = df["Trip Start Timestamp"].dt.hour
if "day_of_week" not in df.columns:
    df["day_of_week"] = df["Trip Start Timestamp"].dt.day_name()

# Downcast numerics to float32 where safe
num_cols = [
    "Trip Miles","Trip Seconds","Additional Charges",
    "pct_low_income","pct_black","pct_hispanic","pct_nonwhite",
    "dropoff_pct_low_income","dropoff_pct_black","dropoff_pct_hispanic","dropoff_pct_nonwhite",
    "pickup_Hardship_index","dropoff_Hardship_index",
    "start_hour","Fare"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")

# Community area IDs as *categorical* (no OHE)
for c in ["Pickup Community Area","Dropoff Community Area"]:
    # Fill missing with a sentinel then cast to category
    df[c] = df[c].astype("Int32").astype("float32").fillna(-1).astype("int32").astype("category")

# Other categoricals
df["day_of_week"] = df["day_of_week"].astype("category")


# -------- 2) Feature matrix (no copy), target --------
feat_cols = [
    # numeric
    "Trip Miles","Trip Seconds","Additional Charges","start_hour",
    "pct_low_income","pct_black","pct_hispanic","pct_nonwhite",
    "dropoff_pct_low_income","dropoff_pct_black","dropoff_pct_hispanic","dropoff_pct_nonwhite",
    "pickup_Hardship_index","dropoff_Hardship_index",
    # categoricals (will be used natively by HGB)
    "day_of_week","Pickup Community Area","Dropoff Community Area",
]
X = df[feat_cols]         # <- view, not a deep copy
y = df["Fare"].astype("float32")

# Drop rows with missing target or critical features to avoid internal copies later
mask = X.notnull().all(axis=1) & y.notnull()
X = X.loc[mask]
y = y.loc[mask]

# -------- 3) Time-based split (avoid leakage) --------
idx_sorted = X.join(df["Trip Start Timestamp"]).sort_values("Trip Start Timestamp").index
cut = int(0.8 * len(idx_sorted))
train_idx = idx_sorted[:cut]
test_idx  = idx_sorted[cut:]

X_train, X_test = X.loc[train_idx], X.loc[test_idx]
y_train, y_test = y.loc[train_idx], y.loc[test_idx]

# -------- 4) Model: HGB with native categoricals + your tuned params --------
# Hint: categorical_features='from_dtype' picks up pandas 'category' columns automatically.
hgb = HistGradientBoostingRegressor(
    max_depth=10,
    max_leaf_nodes=63,
    learning_rate=0.05,
    min_samples_leaf=100,
    l2_regularization=2.0,
    early_stopping=True,
    random_state=42,
    max_bins=255,               
    categorical_features="from_dtype"
)

hgb.fit(X_train, y_train)

# -------- 5) Evaluate --------
pred = hgb.predict(X_test)
rmse = mean_squared_error(y_test, pred, squared=False)
mae  = mean_absolute_error(y_test, pred)
r2   = r2_score(y_test, pred)
print(f"HGB + hardship (native categoricals) | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

# -------- 6)very light permutation importance on a small sample --------

try:
    from sklearn.inspection import permutation_importance
    samp_idx = X_test.sample(n=min(10000, len(X_test)), random_state=42).index
    r = permutation_importance(hgb, X_test.loc[samp_idx], y_test.loc[samp_idx],
                               n_repeats=2, random_state=42, n_jobs=1)
    imp = pd.Series(r.importances_mean, index=hgb.feature_names_in_).sort_values(ascending=False)
    print("\nTop 15 permutation importances:")
    print(imp.head(15).round(4))
    print("\nHardship features:")
    print(imp[[c for c in imp.index if "Hardship" in c]].round(4))
except Exception as e:
    print("Permutation importance skipped:", e)




HGB + hardship (native categoricals) | RMSE: 5.0897 | MAE: 3.2281 | R²: 0.7287

Top 15 permutation importances:
Trip Miles                0.4261
Trip Seconds              0.2731
Additional Charges        0.0652
Dropoff Community Area    0.0541
Pickup Community Area     0.0528
start_hour                0.0194
day_of_week               0.0079
pct_nonwhite              0.0000
pct_low_income            0.0000
dropoff_pct_hispanic      0.0000
pct_hispanic              0.0000
pct_black                 0.0000
dropoff_pct_black         0.0000
dropoff_pct_nonwhite      0.0000
dropoff_Hardship_index    0.0000
dtype: float64

Hardship features:
dropoff_Hardship_index    0.0
pickup_Hardship_index    -0.0
dtype: float64


In [41]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor

def run_hgb(X_train, y_train, X_test, y_test, include_hardship=True):
    # pick features
    base_num = [
        "Trip Miles","Trip Seconds","Additional Charges","start_hour",
        "pct_low_income","pct_black","pct_hispanic","pct_nonwhite",
        "dropoff_pct_low_income","dropoff_pct_black","dropoff_pct_hispanic","dropoff_pct_nonwhite",
    ]
    hardship = ["pickup_Hardship_index","dropoff_Hardship_index"]
    base_cat = ["day_of_week","Pickup Community Area","Dropoff Community Area"]
    
    feats = base_num + base_cat + (hardship if include_hardship else [])
    Xt, Xv = X_train[feats], X_test[feats]
    
    model = HistGradientBoostingRegressor(
        max_depth=10, max_leaf_nodes=63, learning_rate=0.05,
        min_samples_leaf=100, l2_regularization=2.0,
        categorical_features="from_dtype", random_state=42
    )
    model.fit(Xt, y_train)
    preds = model.predict(Xv)
    
    return dict(
        RMSE=mean_squared_error(y_test, preds, squared=False),
        MAE=mean_absolute_error(y_test, preds),
        R2=r2_score(y_test, preds)
    )

# Run both variants
metrics_no_hardship = run_hgb(X_train, y_train, X_test, y_test, include_hardship=False)
metrics_with_hardship = run_hgb(X_train, y_train, X_test, y_test, include_hardship=True)

# Comparison table
comp = pd.DataFrame([metrics_no_hardship, metrics_with_hardship],
                    index=["HGB (no hardship)","HGB (+ hardship)"])
print("\n=== Model performance comparison ===")
print(comp.round(4))



=== Model performance comparison ===
                     RMSE     MAE      R2
HGB (no hardship)  5.0897  3.2281  0.7287
HGB (+ hardship)   5.0897  3.2281  0.7287


In [46]:
def rmse_by_quartile(model, X_test, y_test, hardship_series, n_quartiles=4):
    # Assign trips to hardship-based buckets (trip-weighted)
    q = pd.qcut(hardship_series, q=n_quartiles, labels=[f"Q{i+1}" for i in range(n_quartiles)])
    preds = model.predict(X_test)
    results = []
    for b in q.unique():
        mask = (q == b)
        rmse = mean_squared_error(y_test[mask], preds[mask], squared=False)
        mae  = mean_absolute_error(y_test[mask], preds[mask])
        results.append((b, mask.sum(), rmse, mae))
    return pd.DataFrame(results, columns=["Quartile","n","RMSE","MAE"]).sort_values("Quartile")


best_feats = [
    "Trip Miles","Trip Seconds","Additional Charges","start_hour",
    "pct_low_income","pct_black","pct_hispanic","pct_nonwhite",
    "dropoff_pct_low_income","dropoff_pct_black","dropoff_pct_hispanic","dropoff_pct_nonwhite",
    "pickup_Hardship_index","dropoff_Hardship_index",
    "day_of_week","Pickup Community Area","Dropoff Community Area"
]
model = HistGradientBoostingRegressor(
    max_depth=10, max_leaf_nodes=63, learning_rate=0.05,
    min_samples_leaf=100, l2_regularization=2.0,
    categorical_features="from_dtype", random_state=42
)
model.fit(X_train[best_feats], y_train)

rmse_table = rmse_by_quartile(model, X_test[best_feats], y_test, X_test["pickup_Hardship_index"])
print("\n=== RMSE by pickup hardship quartile (trip-weighted) ===")
print(rmse_table.round(3))



=== RMSE by pickup hardship quartile (trip-weighted) ===
  Quartile       n   RMSE    MAE
3       Q1  709108  5.546  3.805
2       Q2  651665  5.086  3.393
1       Q3  381067  5.895  3.280
0       Q4  574478  3.753  2.295
